In [154]:
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import torch
import math
import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle
from torch.utils.data import TensorDataset, DataLoader, random_split


In [155]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [156]:
load_dotenv()
output_path = os.getenv("VIDEO_OUT")
# Global configuration path
glob_conf_path = '../global_config.py'

In [157]:
train_intensity_df = pd.read_csv(output_path + 'video_data_intensity_train.csv')

In [158]:
train_intensity_df = shuffle(train_intensity_df, random_state=756)

In [159]:
train_intensity_df

,filename,AU01_r_mean,AU01_r_stddevNorm,AU01_r_percentile20.0,AU01_r_percentile50.0,AU01_r_percentile80.0,AU01_r_iqr60_80-20,AU01_r_numPeaks,AU02_r_mean,AU02_r_stddevNorm,...,AU45_r_mean,AU45_r_stddevNorm,AU45_r_percentile20.0,AU45_r_percentile50.0,AU45_r_percentile80.0,AU45_r_iqr60_80-20,AU45_r_numPeaks,video_id,emotion_1_id,group
69,A102_mov_v_4,0.143788,0.438753,0.0,0.000000,0.107118,0.107118,0.617021,0.332805,0.662881,...,0.331346,0.294207,0.0,0.000000,0.234694,0.234694,0.487179,A102,42,2
148,A200_exc_v_4,0.946843,0.108357,0.0,0.000000,0.791292,0.791292,0.063830,0.632974,0.370627,...,0.258488,0.173733,0.0,0.000000,0.257653,0.257653,0.102564,A200,36,1
170,A200_sex_v_4,0.217573,0.385997,0.0,0.000000,0.212163,0.212163,0.127660,0.114591,0.336979,...,0.113977,0.487229,0.0,0.142857,0.055612,0.055612,0.153846,A200,24,1
182,A21_cont_v_4,0.233563,0.112732,0.0,0.083333,0.210781,0.210781,0.085106,0.114064,0.330920,...,0.085829,0.526243,0.0,0.214286,0.048469,0.048469,0.102564,A21,32,2
271,A303_disa_v_4,0.083278,0.554460,0.0,0.166667,0.041465,0.041465,0.127660,0.135620,0.417012,...,0.095552,0.210463,0.0,0.142857,0.071429,0.071429,0.179487,A303,37,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,A101_hop_v_4,0.025572,0.344624,0.0,0.333333,0.013822,0.013822,0.382979,0.008318,0.158984,...,0.108447,0.784598,0.0,0.285714,0.028061,0.028061,0.128205,A101,21,0
143,A200_dist_v_4,0.297589,0.168680,0.0,0.000000,0.269523,0.269523,0.319149,0.184120,0.258822,...,0.113043,0.677841,0.0,0.285714,0.043367,0.043367,0.333333,A200,17,1
293,A303_pri_v_4,0.100238,0.236264,0.0,0.666667,0.079475,0.079475,0.255319,0.240433,0.648165,...,0.211003,0.383440,0.0,0.000000,0.106633,0.106633,0.128205,A303,41,3
224,A227_conf_v_4,0.290741,0.697769,0.0,0.166667,0.121631,0.121631,0.191489,0.875859,0.740910,...,0.441993,0.337053,0.0,0.000000,0.269388,0.269388,0.205128,A227,1,0


In [160]:
# Obtain X, y, and groups from the training dataset
X = train_intensity_df.drop(columns=["filename", "video_id", "emotion_1_id", "group"])
y = train_intensity_df.emotion_1_id

In [161]:
len(y.unique())

44

In [162]:
X = torch.as_tensor(X.values, dtype=torch.float32)
y = torch.as_tensor(y.values, dtype=torch.long)

In [163]:
# create your datset
dataset = TensorDataset(X, y)

train, test = random_split(dataset, [200, 105])

In [164]:
# create your dataloader
train_loader = DataLoader(train, shuffle=True)
test_loader = DataLoader(test, shuffle=True)

In [165]:
seq_model = nn.Sequential(
    nn.Linear(119, 300),
    nn.Dropout(0.5),
    nn.Sigmoid(),
    nn.Linear(300, 300),
    nn.Dropout(0.5),
    nn.Tanh(),
    nn.Linear(300, 44))
seq_model

Sequential(
  (0): Linear(in_features=119, out_features=300, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): Sigmoid()
  (3): Linear(in_features=300, out_features=300, bias=True)
  (4): Dropout(p=0.5, inplace=False)
  (5): Tanh()
  (6): Linear(in_features=300, out_features=44, bias=True)
)

In [166]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lr=1e-3, params=seq_model.parameters(), weight_decay=1e-3)
n_epochs = 50000

In [167]:
data, labels = train_loader.dataset[:]

In [168]:
labels

tensor([11,  0, 29,  8, 22, 11, 28, 23,  4, 38, 34, 32, 22,  1,  8, 27, 38,  9,
        18, 36, 40, 11, 37, 27, 32, 18, 32, 24, 34,  8, 39,  4, 24,  6, 20, 28,
        42, 35,  0, 43, 40,  2, 12, 16, 26, 33, 37, 37, 31,  6, 29, 43, 11, 15,
        38, 13, 19, 24, 30, 38, 17, 35, 12,  0, 33, 24, 21, 30, 27, 18, 26, 29,
        13,  1,  0, 29, 13,  8, 41, 23,  8,  9, 28, 19, 17, 40, 36, 38, 26, 14,
         3, 40, 13,  1, 20, 25, 37, 19, 16, 42, 25, 42, 35,  0, 14, 11, 37,  7,
        30, 34, 35, 10, 38,  9, 10, 34, 19,  3, 15,  8,  2, 42,  3, 39, 15,  1,
        14, 28, 43,  6,  6, 13,  2, 43, 30, 43, 24, 30, 15, 29, 30, 23,  9, 20,
        35, 43,  5, 17, 15, 20, 27, 26, 32, 12, 42,  2, 26,  3,  3,  1, 22, 12,
        34, 14,  5, 27,  7, 27, 33, 20,  2, 26,  3, 29, 11, 18, 41,  5, 14,  7,
        12, 19, 15, 35, 12, 11,  5, 16, 21, 39, 36, 33, 26, 25, 22, 41, 31, 10,
        21,  5])

In [169]:
for epoch in range(1, n_epochs + 1):
    
    data, labels = train_loader.dataset[:]
    
    # forwards pass
    output_train = seq_model(data)

    # calculate loss
    loss_train = criterion(output_train, labels)
    
    
    data, labels = test_loader.dataset[:]
    output_val = seq_model(data)
    loss_val = criterion(output_val, labels)

    
    # set gradients to zero
    optimizer.zero_grad()
    # backwards pass
    loss_train.backward()

    # update model parameters
    optimizer.step()
    if epoch == 1 or epoch % 1000 == 0:
        print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
              f" Validation loss {loss_val.item():.4f}")

Epoch 1, Training loss 3.8039, Validation loss 3.8166
Epoch 1000, Training loss 0.8503, Validation loss 6.7805
Epoch 2000, Training loss 0.3399, Validation loss 8.3591
Epoch 3000, Training loss 0.2427, Validation loss 8.0220
Epoch 4000, Training loss 0.2033, Validation loss 8.0256
Epoch 5000, Training loss 0.2374, Validation loss 8.0340


KeyboardInterrupt: 

In [ ]:
seq_model